<a href="https://colab.research.google.com/github/BenjgLM/streamlit-example/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl
from google.colab import drive
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
from datetime import datetime

# Authentification Google Cloud
auth.authenticate_user()

# Monter Google Drive
drive.mount('/content/drive')

# Initialiser le client BigQuery avec le projet spécifié
client = bigquery.Client(project="ddp-dtm-customer-prd-frlm")

# Exécuter la première requête SQL pour Extract_client
query_client = """
WITH
  sales AS (
  SELECT
    sale.business_unit_id,
    ck_golden_record_id golden_record_id,
    SUM(grand_amount) grand_amount
  FROM
    prod-marketing-datafactory.client_knowledge_view_BU001.ck_golden_record_transaction_line sale
  WHERE
    1=1
    AND ck_golden_record_retail_transaction_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH) AND CURRENT_DATE()
  GROUP BY
    1,
    2 ),
  ref_pro AS (
  SELECT
    golden_record_id,
    ref.ape_code,
    CASE
      WHEN professional_status = 'N/A' THEN 'SUSPECTE'
      ELSE professional_status
  END
    AS professional_status,
    siret_code,
    section_code,
    section_label,
    division_code,
    division_label,
    group_code,
    group_label,
    ape_label
  FROM
    `dfdp-lmfr-data-customer-prod.professional_ref.professional_ref` ref
  INNER JOIN
    `dfdp-lmfr-data-customer-prod.professional_ref.referential_pro_ape`
  USING
    (ape_code) ),
  fid AS (
  SELECT
    ck_golden_record_id golden_record_id,
    loyalty_program_title loyalty_program_label,
    loyalty_level_label,
    loyalty_account_status loyalty_account_status_label,
    loyalty_support_number
  FROM
    `prod-marketing-datafactory.client_knowledge_view_BU001.ck_golden_record_loyalty_info` ),
  seg_rfm AS (
  SELECT
    golden_record_id,
    rfm_segment_code,
    rfm_segment_label
  FROM
    `ddp-dtm-customer-prd-frlm.rfm_segmentation_professional.vf_rfm_segmentation_professional`
  INNER JOIN
    ddp-dtm-customer-int-frlm.rfm_segmentation_professional.vd_rfm_segmentation_label
  USING
    (rfm_segment_code)
  WHERE
    histo_date = DATE_TRUNC(current_date,MONTH) ),
  mag_fav AS (
  SELECT
    ck_golden_record_id golden_record_id,
    concat (criteria_value,
      ' - ',
      store_label) usual_store_id,
    calculation_date
  FROM
    `prod-marketing-datafactory.client_knowledge_view_secured_BU001.ck_golden_record_usual_store_histo_day`
  INNER JOIN
    lmfr-ddp-dwh-int.organization_repository.td_store
  ON
    store_number = criteria_value
  QUALIFY
    ROW_NUMBER() OVER(PARTITION BY golden_record_id ORDER BY calculation_date DESC) = 1 ),
  info_siret AS (
  SELECT
    gr.client_knowledge_golden_record_id golden_record_id,
    client_number,
    last_name,
    first_Name,
    registered_name,
    legal_form
  FROM
    prod-marketing-datafactory.client_knowledge_internal_BU001.golden_record AS gr
  INNER JOIN
    prod-marketing-datafactory.client_knowledge_view_secured_BU001.ck_master_replica AS master
  ON
    master.ck_golden_record_id = gr.client_knowledge_golden_record_id
  LEFT JOIN
    prod-marketing-datafactory.client_knowledge_internal_BU001.golden_record_professional_organization AS orga
  ON
    orga.client_knowledge_golden_record_id = is_member_of
  WHERE
    ck_is_master IS TRUE ),
  optin AS (
  SELECT
    golden_record_id,
    MAX(TOP_OPTIN_Email) optin_email_flag,
    MAX(TOP_OPTIN_SMS ) optin_sms_flag
  FROM (
    SELECT
      DISTINCT ck_golden_record_id golden_record_id,
      CASE
        WHEN communication_type = 'EMAIL' AND value ='ACCEPTED' THEN 1
        ELSE 0
    END
      TOP_OPTIN_Email,
      CASE
        WHEN communication_type = 'SMS' AND value ='ACCEPTED' THEN 1
        ELSE 0
    END
      TOP_OPTIN_SMS
    FROM
      prod-marketing-datafactory.client_knowledge_view_secured_BU001.ck_golden_record_consent
    QUALIFY
      ROW_NUMBER() OVER (PARTITION BY ck_golden_record_id, communication_type ORDER BY submission_date DESC) = 1 )
  GROUP BY
    1 ),
  mail AS (
  SELECT
    client_knowledge_golden_record_id golden_record_id,
    client_number,
    email
  FROM
    `prod-marketing-datafactory.client_knowledge_internal_BU001.golden_record_email` mail ),
  phone AS (
  SELECT
    client_knowledge_golden_record_id golden_record_id,
    client_number,
    phone_number
  FROM
    `prod-marketing-datafactory.client_knowledge_internal_BU001.golden_record_phone` )
SELECT
  info_siret.first_Name prenom,
  info_siret.last_name nom,
  email,
  phone_number numero_telephone,
  sales.golden_record_id,
  siret_code,
  ref_pro.professional_status type_client_pro,
  master.client_number id_client,
  mag_fav.usual_store_id magasin_favori,
  optin.optin_email_flag optin_email,
  optin.optin_sms_flag optin_sms,
  seg_rfm.rfm_segment_label segment_rfm,
  CASE
    WHEN fid.loyalty_program_label IN ('Leroy&moi pour les Pros.', 'Leroy&moi') THEN fid.loyalty_support_number
    ELSE NULL
END
  num_cart_fid,
  CASE
    WHEN fid.loyalty_program_label IN ('Leroy&moi pour les Pros.', 'Leroy&moi') THEN fid.loyalty_program_label
    ELSE NULL
END
  type_prgm_fid,
  CASE
    WHEN fid.loyalty_level_label ='STD' AND fid.loyalty_program_label IN ('Leroy&moi pour les Pros.', 'Leroy&moi') THEN ''
    WHEN fid.loyalty_level_label IN ('LMPRO2',
    'VIP')
  AND fid.loyalty_program_label IN ('Leroy&moi pour les Pros.',
    'Leroy&moi') THEN 'Or'
    WHEN fid.loyalty_level_label IN ('LMPRO1', 'VIP PLUS') AND fid.loyalty_program_label IN ('Leroy&moi pour les Pros.', 'Leroy&moi') THEN 'Platine'
    ELSE NULL
END
  statu_prgm_fid,
  ROUND(sales.grand_amount,2) CA_12mois
FROM
  sales
INNER JOIN
  ref_pro
USING
  (golden_record_id)
LEFT JOIN
  seg_rfm
USING
  (golden_record_id)
LEFT JOIN
  fid
USING
  (golden_record_id)
LEFT JOIN
  mag_fav
USING
  (golden_record_id)
LEFT JOIN
  info_siret
USING
  (golden_record_id)
LEFT JOIN
  optin
USING
  (golden_record_id)
LEFT JOIN
  mail
USING
  (golden_record_id,
    client_number)
LEFT JOIN
  phone
USING
  (golden_record_id,
    client_number)
INNER JOIN (
  SELECT
    *
  FROM
    `prod-marketing-datafactory.client_knowledge_view_secured_BU001.ck_master_replica`
  WHERE
    ck_is_master IS TRUE ) master
ON
  sales.golden_record_id =master.ck_golden_record_id
"""

# Exécuter la requête SQL
query_job_client = client.query(query_client)

# Télécharger les résultats en tant que DataFrame
df_client = query_job_client.to_dataframe()

# Obtenir la date actuelle
current_date = datetime.now().strftime("%Y-%m-%d")

# Spécifier le chemin du dossier dans Google Drive pour le fichier XLSX avec la date ajoutée
drive_path_client = f"/content/drive/Shareddrives/Conquête des PROS - équipe projet/Pilotage/CRM/Import DATA/Extract_client_{current_date}.xlsx"

# Sauvegarder le DataFrame en XLSX dans le dossier spécifié
df_client.to_excel(drive_path_client, index=False)

print(f"Les résultats ont été sauvegardés dans {drive_path_client} sous forme de fichier XLSX.")

# Exécuter la deuxième requête SQL pour Extract_entreprise
query_entreprise = """
WITH
  list_cli_actif AS (
  SELECT
    *,
    COUNT(*) OVER (PARTITION BY siret_code ) AS siret_count
  FROM (
    SELECT
      DISTINCT ref.golden_record_id,
      is_member_of,
      siret_code,
      loyalty_account_id,
      loyalty_program_title,
      CASE
        WHEN fid.loyalty_level_label ='STD' THEN ''
        WHEN fid.loyalty_level_label IN ('LMPRO2',
        'VIP') THEN 'Or'
        WHEN fid.loyalty_level_label IN ('LMPRO1', 'VIP PLUS') THEN 'Platine'
        ELSE NULL
    END
      AS loyalty_level_label,
      loyalty_account_is_owner,
      loyalty_account_status,
      ape.ape_label
    FROM
      prod-marketing-datafactory.client_knowledge_view_BU001.ck_golden_record_transaction_line sale
    INNER JOIN
      `dfdp-lmfr-data-customer-prod.professional_ref.professional_ref` ref
    ON
      ck_golden_record_id=golden_record_id
    INNER JOIN
      `dfdp-lmfr-data-customer-prod.professional_ref.referential_pro_ape` ape
    USING
      (ape_code)
    LEFT JOIN
      prod-marketing-datafactory.client_knowledge_internal_BU001.golden_record
    ON
      client_knowledge_golden_record_id= golden_record_id
    LEFT JOIN
      prod-marketing-datafactory.client_knowledge_view_BU001.ck_golden_record_loyalty_info fid
    ON
      sale.ck_golden_record_id=fid.ck_golden_record_id
    LEFT JOIN (
      SELECT
        ck_golden_record_id golden_record_id,
        criteria_value usual_store_id,
        calculation_date
      FROM
        `prod-marketing-datafactory.client_knowledge_view_secured_BU001.ck_golden_record_usual_store_histo_day`
      INNER JOIN
        lmfr-ddp-dwh-int.organization_repository.td_store
      ON
        store_number = criteria_value
      QUALIFY
        ROW_NUMBER() OVER(PARTITION BY golden_record_id ORDER BY calculation_date DESC) = 1 ) fav
    ON
      fav.golden_record_id=sale.ck_golden_record_id
    WHERE
      1=1
      AND ck_golden_record_retail_transaction_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 12 MONTH)
      AND CURRENT_DATE()
      /*AND fav.usual_store_id IN (12,
        115,
        63,
        21,
        29,
        45,
        146,
        156,
        173,
        37)*/
         ) ),
  base_siren AS (
  SELECT
    DISTINCT establishment_usual_name,
    address,
    establishment_postal_code,
    establishment_city_label,
    siret,
    NULL AS TVA,
    NULL AS num_IDE,
    establishment_main_activity
  FROM
    ddp-dtm-customer-prd-frlm.firm_open_data.vf_french_firm ),
  firm_mdf AS (
  SELECT
    DISTINCT adm.administrative_code,
    orga.registered_name
  FROM
    `prod-marketing-datafactory.client_knowledge_internal_BU001.golden_record_administrative_code` adm
  INNER JOIN
    list_cli_actif
  ON
    is_member_of = client_knowledge_golden_record_id
  LEFT JOIN
    `prod-marketing-datafactory.client_knowledge_internal_BU001.golden_record_professional_organization` orga
  USING
    (client_knowledge_golden_record_id)
  QUALIFY
    ROW_NUMBER() OVER (PARTITION BY adm.administrative_code ORDER BY orga.source_last_update_date DESC) = 1 )
SELECT
  DISTINCT
  CASE
    WHEN (establishment_usual_name IS NULL OR establishment_usual_name = '') THEN registered_name
    ELSE establishment_usual_name
END
  AS nom_entreprise,
  address AS adresse,
  establishment_postal_code AS code_postale,
  establishment_city_label AS ville,
  siret,
  establishment_main_activity AS code_ape,
  ape_label AS libelle_ape,
  CASE
    WHEN siret_count > 1 THEN NULL
    WHEN siret_count = 1
  AND loyalty_program_title IN ('Leroy&moi pour les Pros.',
    'Leroy&moi') THEN loyalty_account_id
    ELSE NULL
END
  AS num_compte_fid,
  CASE
    WHEN siret_count > 1 THEN NULL
    WHEN siret_count = 1
  AND loyalty_program_title IN ('Leroy&moi pour les Pros.',
    'Leroy&moi') THEN loyalty_program_title
    ELSE NULL
END
  AS type_prgm_fid,
  CASE
    WHEN siret_count > 1 THEN NULL
    WHEN siret_count = 1
  AND loyalty_program_title IN ('Leroy&moi pour les Pros.',
    'Leroy&moi') THEN loyalty_level_label
    ELSE NULL
END
  AS statu_prgm_fid
FROM
  base_siren
INNER JOIN
  list_cli_actif ref
ON
  siret = siret_code
LEFT JOIN
  firm_mdf
ON
  administrative_code = siret
"""

# Exécuter la requête SQL
query_job_entreprise = client.query(query_entreprise)

# Télécharger les résultats en tant que DataFrame
df_entreprise = query_job_entreprise.to_dataframe()

# Spécifier le chemin du dossier dans Google Drive pour le fichier XLSX avec la date ajoutée
drive_path_entreprise = f"/content/drive/Shareddrives/Conquête des PROS - équipe projet/Pilotage/CRM/Import DATA/Extract_entreprise_{current_date}.xlsx"

# Sauvegarder le DataFrame en XLSX dans le dossier spécifié
df_entreprise.to_excel(drive_path_entreprise, index=False)

print(f"Les résultats ont été sauvegardés dans {drive_path_entreprise} sous forme de fichier XLSX.")


Mounted at /content/drive
Les résultats ont été sauvegardés dans /content/drive/Shareddrives/Conquête des PROS - équipe projet/Pilotage/CRM/Import DATA/Extract_client_2024-08-20.xlsx sous forme de fichier XLSX.
Les résultats ont été sauvegardés dans /content/drive/Shareddrives/Conquête des PROS - équipe projet/Pilotage/CRM/Import DATA/Extract_entreprise_2024-08-20.xlsx sous forme de fichier XLSX.
